# Музыка Больших Городов

В наличии были следующие данные о прослушиваниях треков в сервисе Яндекс.Музыка:

* Название трека
* Исполнитель
* Жанр
* Город
* День прослушивания
* Время прослушивания

Задача:

Провести сравнение музыкальных предпочтений у пользователей Яндекс.Музыки, проживающих в Москве и Санкт-Петербурге. Сравнить прослушивания в понедельник, среду и пятницу по городам, а также сравнить жанры, которые слушают пользователи утром в понедельник и вечером в пятницу в Москве и Санкт-Петербурге.

## Импорт библиотек

In [1]:
import pandas as pd

Чтение файла `music_project.csv` и его сохранение в переменную df

In [2]:
df = pd.read_csv('/datasets/music_project.csv')

Вывожу первые 10 строк таблицы

In [39]:
df.head(10)

,user_id,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,unknown,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,unknown,dance,Saint-Petersburg,21:20:49,Wednesday


Вывожу информацию о таблице

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


В таблице семь столбцов. Тип данных во всех столбцах — `object`.

Подробная информация какой столбец и какую информацию он содержит
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

Количество значений в столбцах различается. Значит, в данных есть значения `None`

**Выводы**

Каждая строка таблицы содержит информацию о композициях определённого жанра в определённом исполнении, которые пользователи слушали в одном из городов в определённое время и день недели. Две проблемы, которые нужно решать: пропуски и некачественные названия столбцов. Для проверки рабочих гипотез особенно ценны столбцы time, day и City. Данные из столбца genre позволят узнать самые популярные жанры.

## Предобработка данных

Исключим пропуски, переименуем столбцы, а также проверим данные на наличие дубликатов.

In [6]:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')


Названия некотрых колонок, затруднят доступ данных, поэтому приведем все названия в правильный вид

In [7]:
df = df.rename(columns={'  userID': 'user_id'
                        , 'Track': 'track'
                        ,'  City  ': 'city'
                        , 'Day': 'day'})

In [8]:
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')


В данных были обнаружены пропуски, посчитаем их количесто по столбцам

In [9]:
missed_num = df.isna().sum()
missed_num

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64


Пустые значения свидетельствуют, что для некоторых треков доступна не вся информация. Причины могут быть разные, поэтому заменяем пропущенные значения в столбцах на строку 'unknown'. После этой операции нужно убедиться, что таблица больше не содержит пропусков.

In [10]:
columns_to_replace = ['track', 'artist', 'genre']
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')

In [11]:
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64


Подсчет явных дубликатов

In [12]:
df.duplicated().sum()

3826


Использую функцию `drop_duplicates()` что бы от них избавиться

In [13]:
df = df.drop_duplicates()

In [14]:
df.duplicated().sum()

0


Явных дубликатов больше не осталось

Теперь избавимся от неявных дубликатов в колонке `genre`

Отсортируем в алфавитном порядке и выведем все уникальные значения из столбца `genre`

In [15]:
df_sorted = df.sort_values(by='genre')
df_unique = df_sorted['genre'].unique()
df_unique

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'folkrock' 

В списке обнаружены не явные дубликаты названия `hiphop`:

* *hip*,
* *hop*,
* *hip-hop*.

Используя метод `replace()` исправлю их на значение `hiphop`

In [16]:
# Устранение неявных дубликатов
incorrect_genres = ['hip', 'hop', 'hip-hop']
for i in incorrect_genres:
    df['genre'] = df['genre'].replace(i, 'hiphop')

In [ ]:
# Проверка на неявные дубликаты
srt = df.sort_values(by='genre')
srt = srt['genre'].unique()
print(srt)

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'folkrock' 

**Выводы**

На этапе предобработки в данных обнаружились не только пропуски и проблемы с названиями столбцов, но и всяческие виды дубликатов. Их удаление позволит провести анализ точнее. Поскольку сведения о жанрах важно сохранить для анализа, не просто удаляем все пропущенные значения, но заполним пропущенные имена исполнителей и названия треков. Имена столбцов теперь корректны и удобны для дальнейшей работы.

## Сравнения прослушивания музыки в разных городах

Была выдвинута гипотеза, что в Москве и Санкт-Петербурге пользователи слушают музыку по-разному. Проверяем это предположение по данным о трёх днях недели — понедельнике, среде и пятнице.

Для каждого города устанавливаем количество прослушанных в эти дни композиций с известным жанром, и сравниваем результаты.

In [18]:
# Подсчёт прослушиваний в каждом городе
city_grouped_df = df.groupby('city')['time'].count()
city_grouped_df

city
Moscow              42741
Saint-Petersburg    18512
Name: time, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

In [19]:
# Подсчёт прослушиваний в каждый из трёх дней
day_grouped_df = df.groupby('day')['time'].count()
day_grouped_df

day
Friday       21840
Monday       21354
Wednesday    18059
Name: time, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

In [20]:
# Создаю функцию number_tracks()
def number_tracks(day:str, city:str):
    """ Функция для подсчета кол-ва треков прослушанный в заданном городе и дне
    
    :param day:str - день недели
    :param city:str - город
    :return: количество прослушиваний
    """

    track_list = df[df['day']==day]
    track_list = track_list[track_list['city']==city]
    track_list_count = track_list['user_id'].count()
    return track_list_count

Вызываю функцию 6 раз, что бы найти кол-во прослушиваний для каждого города (Москва, Санкт-Петербург) для дней (понедельник, среда, пятница)

In [21]:
# количество прослушиваний в Москве по понедельникам
number_tracks('Monday', 'Moscow')

15740

In [22]:
# количество прослушиваний в Санкт-Петербурге по понедельникам
number_tracks('Monday', 'Saint-Petersburg')

5614

In [23]:
# количество прослушиваний в Москве по средам
number_tracks('Wednesday', 'Moscow')

11056

In [24]:
# количество прослушиваний в Санкт-Петербурге по средам
number_tracks('Wednesday', 'Saint-Petersburg')

7003

In [25]:
# количество прослушиваний в Москве по пятницам
number_tracks('Friday', 'Moscow')

15945

In [26]:
# количество прослушиваний в Санкт-Петербурге по пятницам
number_tracks('Friday', 'Saint-Petersburg')

5895

Создаю DataFrame с данными которые я получил

In [27]:
# Таблица с результатами
data = [
    ['Moscow', 16715, 11755, 16890],
    ['Saint-Petersburg', 5982, 7478, 6259]
]
columns = ['city', 'monday', 'wednesday', 'friday']
pd.DataFrame(data=data, columns=columns)

,city,monday,wednesday,friday
0,Moscow,16715,11755,16890
1,Saint-Petersburg,5982,7478,6259


**Выводы**

Результаты показывают, что относительно среды музыку в Петербурге и Москве слушают «зеркально»: в Москве пики приходятся на понедельник и пятницу, а в среду время прослушивания снижается. Тогда как в Санкт-Петербурге среда — день самого большого интереса к музыке, а в понедельник и пятницу он меньше, причём почти одинаково меньше.

### Музыка в начале и в конце недели

Создаю два DataFrame:
* `moscow_general` - все данные по Москве
* `spg_general` - все данные по Санкт-Петербургу

In [28]:
moscow_general = df[df['city']=='Moscow']
moscow_general.shape

(42741, 7)


In [29]:
spb_general = df[df['city']=='Saint-Petersburg']
spb_general.shape

(18512, 7)


Создайте функцию `genre_weekday()` с четырьмя параметрами что бы найти топ-10 жанров по количеству прослушиваний в заданные день и промежуток времени

In [30]:
def genre_weekday(df, day, time1, time2):
    """
    :param df:pd.DataFrame - DataFrame в котором будут проводиться действия
    :param day:str - день недели
    :param time1
    :param time2
    :return:pd.Series - 10 самых популярных жанров в указанный отрезок
    """

    genre_df = df[df['day'] == day]
    genre_df = genre_df[genre_df['time'] > time1]
    genre_df = genre_df[genre_df['time'] < time2]
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)

    return genre_df_sorted[:10]
moscow_general.shape
spb_general.shape

(42741, 7)
(18512, 7)


In [31]:
# вызов функции для утра понедельника в Москве
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64


In [32]:
# вызов функции для утра понедельника в Петербурге
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64


In [33]:
# вызов функции для вечера пятницы в Москве
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64


In [34]:
# вызов функции для вечера пятницы в Петербурге
print(genre_weekday(spb_general, 'Friday', '17:00', '23:00'))
print(moscow_general.shape)
print(spb_general.shape)

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64
(42741, 7)
(18512, 7)


**Вывод**

В Москве, кроме абсолютно популярного жанра поп, есть направление русской популярной музыки. Значит, что интерес к этому жанру шире. А рэп, вопреки предположению, занимает в обоих городах близкие позиции.

### Сравнения предпочтений в жанрах между Москвой и Петербургом

Создаю две переменные для Москвы и Питера, в каждом будет количество прослушиваний по каждому жанру

In [35]:
moscow_general_grouped = moscow_general.groupby('genre')['genre'].count()
moscow_genres = moscow_general_grouped.sort_values(ascending=False)

In [36]:
# просмотр первых 10 строк moscow_genres
moscow_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64


In [37]:
spb_general_grouped = spb_general.groupby('genre')['genre'].count()
spb_genres = spb_general_grouped.sort_values(ascending=False)

In [38]:
# просмотр первых 10 строк spb_genres
print(spb_genres.head(10))

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64


**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Рабочие гипотезы:

* музыку в двух городах — Москве и Санкт-Петербурге — слушают в разном режиме;

* списки десяти самых популярных жанров утром в понедельник и вечером в пятницу имеют характерные отличия;

* население двух городов предпочитает разные музыкальные жанры.

Общие результаты

Москва и Петербург сходятся во вкусах. При этом зависимости предпочтений от дня недели в каждом отдельном городе нет — люди постоянно слушают то, что им нравится. Но между городами в разрезе дней неделей наблюдается зеркальность относительно среды: Москва больше слушает в понедельник и пятницу, а Петербург наоборот - больше в среду, но меньше в понедельник и пятницу.

В результате первая гипотеза подтверждена, вторая гипотеза подтверждена и третья гипотеза подтверждена.